## 필요한 라이브러리 Import

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re



## 네이버증권 Financial Summary 크롤링

기본적으로 네이버증권은 URL에 종목코드륿 변수로 받아 종목코드별(기업별) 페이지를 구성하고 있다.<br><br>
Chrome 개발자 도구에 들어가 Network를 켜보면 "cF1001.aspx?cmp=cd"부분을 보면 매출액, 영업이익 등이 html 언어로 표현되어 있는 것을 알 수 있다. <br><br>
URL을 살펴보면 cmp_cd는 종목코드, freq_typ은 연간(A), 분기(Q), encparam과 id는 종목별로 랜덤하게 부여되는 key값 정도로 파악된다. <br><br>
encparam과 id는 랜덤 값으로 파악되기 때문에 이 정보를 알고 접근할 수는 없다. 그래서 2개의 값을 html에서 찾아서 접근하는 방법을 사용하기로 했다.

## encparam과 id 값 찾기

In [ ]:
def get_basic_key_values(code):
    # 종목별 encparam, encid 찾기
    url = f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={code}'
    html = requests.get(url).text
    
    re_enc = re.compile("encparam: '(.*)'", re.IGNORECASE)
    re_id = re.compile("id: '([a-zA-Z0-9]*)' ?", re.IGNORECASE)
    
    encparam = re_enc.search(html).group(1)
    encid = re_id.search(html).group(1)
    
    return encparam, encid

get_basic_key_values('005930') # 삼성전자

## get_basic_key_values 함수를 사용해서 Financial Summary 값을 데이터 프레임으로 불러오기

In [17]:
def get_dataframe(code):
    
    encparam, encid = get_basic_key_values(code)

    url = f'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd={code}&fin_type=0&freq_type=A&encparam={encparam}&id={encid}'
    headers = {'Referer': 'HACK'}
    html = requests.get(url, headers=headers).text
    dfs = pd.read_html(html)
    df = dfs[1]

    return df

get_dataframe('005930') # 삼성전자